In [16]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors 
import pandas as pd 
import seaborn as sns
import random

## <font color=darkred> Dataset </font>

We provide below two datasets (one to train each model and another one to test each model).

Datasets contain input features (meteorological data) and output (kWh consumptions) to be predicted from the inputs.

In [17]:
df_train = pd.read_csv('data_train.csv')
df_train.head() 

,temperature,humidity,horizontal_radiation,normal_radiation,heating,cooling,ventilation,gains
0,4.93,65.34,0.0,0.0,171.65,0.0,0,138.6
1,5.07,54.61,0.0,0.0,170.59,0.0,0,138.6
2,3.71,62.33,0.0,0.0,206.19,0.0,0,138.6
3,2.36,70.05,0.0,0.0,241.49,0.0,0,138.6
4,1.00,77.77,0.0,0.0,275.97,0.0,0,138.6


In [18]:
df_train.shape

(17520, 8)

In [19]:
df_test =pd.read_csv('data_test.csv')
df_test.head()

,temperature,humidity,horizontal_radiation,normal_radiation,heating,cooling,ventilation,gains
0,7.09,88.42,0.0,0.0,0.00,0.0,0,138.6
1,4.00,88.58,0.0,0.0,7.18,0.0,0,138.6
2,2.23,87.25,0.0,0.0,125.72,0.0,0,138.6
3,2.52,87.17,0.0,0.0,106.85,0.0,0,138.6
4,2.72,89.17,0.0,0.0,104.25,0.0,0,138.6


#### Build input/output from dataframe

In [20]:
labels_output = ['heating','cooling','gains','ventilation']
labels_input = ['temperature','humidity','normal_radiation','horizontal_radiation']

In [21]:
def get_input_output(df,labels_usage,labels_input):
    dfX = df[labels_input]
    dfY = df[labels_usage]
    return dfX, dfY

In [22]:
dfX_train, dfY_train = get_input_output(df_train,labels_output,labels_input)
dfX_test, dfY_test = get_input_output(df_test,labels_output,labels_input)

In [23]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

In [24]:
def scaler_fit_transform(d_train,d_test):
    scaler = MinMaxScaler()
    data_train_sc = scaler.fit_transform(d_train) 
    data_test_sc = scaler.transform(d_test)
    return scaler, data_train_sc, data_test_sc

In [25]:
scaler_x, X_train_sc, X_test_sc = scaler_fit_transform(dfX_train,dfX_test) 
scaler_y, y_train_sc, y_test_sc = scaler_fit_transform(dfY_train,dfY_test) 

In [27]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
class TSDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, seq_len=1):
        self.X = X
        self.y = y
        self.seq_len = seq_len

    def __len__(self):
        return self.X.__len__() - (self.seq_len-1)

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index:index+self.seq_len])

In [28]:
# We define a problem where we want to predict 1 week (7*24h) of output from the input
seq_len = 24*7
# Each gradient step uses batch_size sequences of input/output
batch_size = 16

In [29]:
def set_loader(X_train,y_train,X_test,y_test,seq_len,batch_size): 
    train_dataset = TSDataset(torch.Tensor(X_train), torch.Tensor(y_train), seq_len)
    test_dataset = TSDataset(torch.Tensor(X_test), torch.Tensor(y_test), seq_len)
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False, drop_last=True)
    return train_dataset, test_dataset, train_loader, test_loader

In [30]:
train_dataset, test_dataset, train_loader, test_loader = set_loader(X_train_sc,y_train_sc,X_test_sc,y_test_sc,seq_len,batch_size)

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim

## <font color=darkred> Models </font>

### Recurrent architectures 

 - Complete the following RNNmodel class

In [ ]:
class RNNmodel(nn.Module):
    def __init__(self, input_d, hidden_d, layer_n, output_d, dropout_p):
        """
        input_d (int): dimension of the input layer
        hidden_d (int): dimension of each layer
        layer_n (int): number of layers
        output_d (int): dimension of output layer
        dropout_p (float): probability of dropping neurons
        """
        super(RNNmodel, self).__init__()

        self.hidden_d = hidden_d
        self.layer_n = layer_n
        
        # complete the following RNN module
        # TO DO
        # self.rnn = 
        # Add a linear output for the regression problem
        # TO DO
        # self.fc = 

    def forward(self, x):
        """
        x (torch.Tensor): input tensor (batch size, sequence length, input_d)
        """

        # Forward pass
        # call the RNN module to compute the output and the hidden state of the RNN fed with x
        # TO DO
        # out, h0 = 

        out = self.fc(out)
        return out

 - Complete similarly the following LSTMmodel class

In [ ]:
class LSTMmodel(nn.Module):
    def __init__(self, input_d, hidden_d, layer_n, output_d, dropout_p):
        """
        input_d (int): dimension of the input layer
        hidden_d (int): dimension of each layer
        layer_n (int): number of layers
        output_d (int): dimension of output layer
        dropout_p (float): probability of dropping neurons
        """
        super(LSTMmodel, self).__init__()

        self.hidden_d = hidden_d
        self.layer_d = layer_n

        # complete the LSTM module
        # TO DO 
        

    def forward(self, x):
        """
        x (torch.Tensor): input tensor (batch size, sequence length, input_d)
        """

        # Complete the forward pass 
        # TO DO
    

        return out

 - Provide similarly a GRUmodel class

In [ ]:
# TO DO

## <font color=darkred> Optimization </font>

 - Complete the following class to optimize all models

In [ ]:
class Optimization:
    def __init__(self, model, loss_fn, optimizer, device):
        """
        model: selected model 
        loss_fn: loss function
        optimizer: function to optimize loss
        """
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = device
        self.train_losses = []
        self.val_losses = []

    def train_step(self, x, y):
        """
        x: input data
        y: target data
        """

        self.model.train()

        # Compute the prediction provided by model 
        # TO DO
        # yhat = 

        # Compute loss
        # TO DO
        #loss = 
        
        # Gradient for the optimization
        loss.backward()

        # Updates parameters
        self.optimizer.step()
        self.optimizer.zero_grad()

        return loss.item()

    def train(self, train_loader, val_loader, batch_size=64, n_epochs=50, n_features=1):
        """
        train_loader: training data
        val_loader: validation data
        batch_size: batch size
        n_epochs: number of epochs
        n_features: number of features
        """
        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.to(self.device)
                y_batch = y_batch.to(self.device)
                # Compute the loss using train_step on the current batch
                # TO DO
                #loss = 
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.to(self.device)
                    y_val = y_val.to(self.device)
                    self.model.eval()
                    # Compute the loss function on the current validation batch
                    # TO DO
                    # yhat = 
                    # val_loss = 
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)


                print(f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}")

    def evaluate(self, test_loader, batch_size=1, n_features=1):
        """
        test_loader: dataLoader
        batch_size: batch size
        n_features: number of features
        """
    
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.to(self.device)
                y_test = y_test.to(self.device)

                self.model.eval()
                yhat = self.model(x_test)

                yhat=yhat.cpu().data.numpy()
                predictions.append(yhat)
                y_test=y_test.cpu().data.numpy()
                values.append(y_test)

        return predictions, values

    def plot_losses(self):
        # Plot train_losses and val_losses (to be called after training)
        # TO DO


## <font color=darkred> Training </font>

In [ ]:
# parameters of the neural networks
# input dimension
input_d = 4
# output dimension
output_d = 4
# hidden layers dimension
hidden_d = 8
#number of layers
layer_n = 2
# number of times all data are processed during training
n_epochs = 10
# parameters for the optimizer
lrate = 1e-4
weight_decay = 1e-6 

In [ ]:
model_params = {'input_d': input_d,
                'hidden_d' : hidden_d,
                'layer_n' : layer_n,
                'output_d' : output_d}

In [ ]:
def get_model(model, model_params):
    models = {"rnn": RNNmodel,"lstm": LSTMmodel,"gru": GRUmodel}
    return models.get(model.lower())(**model_params)

In [ ]:
# default loss function: MSE
loss_fn = nn.MSELoss(reduction="mean")
device = "cuda" if torch.cuda.is_available() else "cpu"

 - Complete the following cell to train a RNN

In [ ]:
model_name = 'rnn'
# get model 
# TO DO
# model =  
# Set optimizer
optimizer = optim.Adam(model.parameters(), lr=lrate, weight_decay=weight_decay)
# Call the optimization class, train opt_rnn and plot losses
# TO DO 
# opt_rnn = 
# opt_rnn.
# opt_rnn.

 - Analyze the influence of the learning rate of the optimizer and of the batch size on the test and train losses

In [ ]:
# TO DO

 - Train a LSTM and a GRU model

In [ ]:
# TO DO

- Compute the MSE for each model on the test set  and display some predictions to provide qualitative results

In [ ]:
# TO DO

 - Perform a cross validation to get the results of the best model (you can optimize the number of layers and the hidden dimension for instance).

 - CNN can also be used to model time series, in particular with one dimensional convolutional layers.
 
 - These layers are defined by ``nn.Conv1d`` in Torch with main arguments ``in_channels``, ``out_channels``, ``kernel_size`` (an integer in the one dimensional setting), ``stride`` and ``padding``.
 
 - Build a CNNmodel class and compare its performance with previous recurrent models.

In [ ]:
# TO DO

 - (**Bonus**) Implement a (conditional) VAE to train the distribution of the ouput

In [ ]:
#TO DO